In [1]:
!date

Thu Jan 12 01:56:32 PST 2023


In [2]:
!echo $CONDA_PREFIX

/c4/home/derek/miniconda3/envs/py39


In [4]:
import HTSeq
import numpy
from matplotlib import pyplot

In [21]:
%%time
bamfile = HTSeq.BAM_Reader( "/c4/home/derek/data1/HDD-2/derek/SLR_temp/CAT_20220904_PBMC/sample_1/merged/merged_complete_5prime_downsample.bam" )
gtffile = HTSeq.GFF_Reader( "/c4/home/derek/data1/HDD-2/derek/reference/human_hp3_reference/genes/genes.gtf" )
# halfwinwidth = 3000
# fragmentsize = 200




CPU times: user 102 ms, sys: 59 ms, total: 161 ms
Wall time: 1.23 s


In [10]:
gtffile

<GFF_Reader object, connected to file name '/c4/home/derek/data1/HDD-2/derek/reference/human_hp3_reference/genes/genes.gtf'>

In [ ]:
#get genes from GTF
genes = set()
for feature in gtffile:
    if feature.type == "gene":
        genes.add(feature.iv)

In [36]:
%%time
#gene_bodies = HTSeq.GenomicArrayOfSets( "auto", stranded=False )

        
profile = numpy.zeros( 1000, dtype='i' )
for p in genes:
    window = HTSeq.GenomicInterval( p.chrom, p.start_as_pos.pos, 
                                   p.end_as_pos.pos, "." )
    for almnt in bamfile[ window ]:
        #print('test')
        if p.strand == "+":
            start_in_window = almnt.iv.start 
            end_in_window   = almnt.iv.end   
        else:
            start_in_window =  almnt.iv.end
            end_in_window   =  almnt.iv.start
            
        
        profile[ start_in_window : end_in_window ] += 1

 
        # window = HTSeq.GenomicInterval( start_p.chrom, start_p.pos, end_p.pos, "." )
        # tsspos[ window ] += start_p



KeyboardInterrupt: 

In [34]:
profile

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
profile = numpy.zeros( 2*halfwinwidth, dtype="i" )
for almnt in bamfile:
   if almnt.aligned:
      almnt.iv.length = fragmentsize
      s = set()
      for step_iv, step_set in tsspos[ almnt.iv ].steps():
         s |= step_set
      for p in s:
         if p.strand == "+":
            start_in_window = almnt.iv.start - p.pos + halfwinwidth
            end_in_window   = almnt.iv.end   - p.pos + halfwinwidth
         else:
            start_in_window = p.pos + halfwinwidth - almnt.iv.end
            end_in_window   = p.pos + halfwinwidth - almnt.iv.start
         start_in_window = max( start_in_window, 0 )
         end_in_window = min( end_in_window, 2*halfwinwidth )
         profile[ start_in_window : end_in_window ] += 1